In [49]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

import tensorflow
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Input, Dense, Dropout
import keras_tuner as kt

In [50]:
df = pd.read_csv("/content/diabetes.csv")
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [51]:
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values

In [52]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [53]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [54]:
model = Sequential([
    Input(shape=(8,)),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_2 (Dense)                 │ (None, 32)             │           288 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 321 (1.25 KB)

 Trainable params: 321 (1.25 KB)

 Non-trainable params: 0 (0.00 B)

In [55]:
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [56]:
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs = 100)

Epoch 1/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 42ms/step - accuracy: 0.7157 - loss: 0.6375 - val_accuracy: 0.6948 - val_loss: 0.6075
Epoch 2/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7172 - loss: 0.5890 - val_accuracy: 0.7208 - val_loss: 0.5802
Epoch 3/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7354 - loss: 0.5492 - val_accuracy: 0.7532 - val_loss: 0.5604
Epoch 4/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7547 - loss: 0.5249 - val_accuracy: 0.7468 - val_loss: 0.5469
Epoch 5/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7277 - loss: 0.5259 - val_accuracy: 0.7597 - val_loss: 0.5378
Epoch 6/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7286 - loss: 0.5168 - val_accuracy: 0.7597 - val_loss: 0.5308
Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7530 - loss: 0.4748 - val_accuracy: 0.7532 - val_loss: 0.5237
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7603 - loss: 0.4885 - val_accuracy: 0.7532 - 

In [68]:
# This syntex suggested by keras
def build_model(hp):
    # Building model architecture
    model = Sequential([
        Input(shape=(8,)),
        Dense(32, activation='relu'),
        Dense(1, activation='sigmoid')
    ])

    # Creating your parameters grid - Create an object wrt every tuning parameter
    optimizer = hp.Choice(
        'optimizer', # Name of what you are tuning
        values = ['adam', 'sgd', 'rmsprop', 'adadelta']
    )

    # Compiling the model
    model.compile(optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])

    # Returning the model
    return model

# This is RandomSearchCV of sklearn in keras
tuner = kt.RandomSearch(
    build_model, # Your function
    objective = 'val_accuracy', # What you want to maximize(metrics) or minimize(loss functions)
    max_trials = 5, # Randomly select 5 sets from hyperparameter search space randomly
    directory = "Hyperparameter Tuning",
    project_name = "optimizer"
)

# Fitting and Searching
tuner.search(X_train, y_train, validation_data = (X_test, y_test), epochs = 5)

Reloading Tuner from Hyperparameter Tuning/optimizer/tuner0.json


In [69]:
tuner.search_space_summary()

Search space summary
Default search space size: 1
optimizer (Choice)
{'default': 'adam', 'conditions': [], 'values': ['adam', 'sgd', 'rmsprop', 'adadelta'], 'ordered': False}


In [70]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'rmsprop'}

In [71]:
model = tuner.get_best_models(num_models = 1)[0] # Top 1st model

/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 6 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


What are the implications?

- For Inference (Prediction): If you are just loading the model to make predictions, this warning is usually harmless. The model's weights (the actual learned parameters of your neural network) are typically loaded correctly. The optimizer's state is only relevant for training.

- For Resuming Training: If you intend to continue training the model from where it left off, this is a more significant warning. Because the optimizer's state couldn't be loaded, the optimizer will be re-initialized from scratch. This means it will lose its accumulated momentum, adaptive learning rates, or other internal states. This can sometimes lead to:

    - A temporary dip in performance at the start of resumed training.

    - Slower convergence in the initial epochs of resumed training.

    - In rare cases, instability if the new initial optimizer state is very different from what the training process was expecting.

If you need to resume training:

- Ensure consistent model architecture: The most robust solution is to ensure that the model architecture (the layers and their configurations) is identical to the one that was saved. If you changed the model, you effectively have a "new" model, and the old optimizer state doesn't apply.

- Save/Load Weights Only: Instead of saving the entire model, you can save only the model's weights and then load them into a newly created model with the correct optimizer for your current architecture.

In [72]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 32)             │           288 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 321 (1.25 KB)

 Trainable params: 321 (1.25 KB)

 Non-trainable params: 0 (0.00 B)

In [73]:
history = model.fit(X_train, y_train, validation_data = (X_test, y_test), epochs = 100, initial_epoch = 5)
# initial_epoch - The model is alread trained for 5 epochs, we are starting the training from 6 onwards

Epoch 6/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 37ms/step - accuracy: 0.7863 - loss: 0.4861 - val_accuracy: 0.7532 - val_loss: 0.5186
Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7658 - loss: 0.4826 - val_accuracy: 0.7597 - val_loss: 0.5152
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7299 - loss: 0.4992 - val_accuracy: 0.7662 - val_loss: 0.5117
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7801 - loss: 0.4721 - val_accuracy: 0.7662 - val_loss: 0.5113
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7628 - loss: 0.4645 - val_accuracy: 0.7727 - val_loss: 0.5114
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7722 - loss: 0.4597 - val_accuracy: 0.7727 - val_loss: 0.5112
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7423 - loss: 0.4721 - val_accuracy: 0.7727 - val_loss: 0.5090
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7576 - loss: 0.4642 - val_accuracy: 0.779

In [74]:
# Number of neurons in a perticular layer
def build_model(hp):
    # Building model architecture
    model = Sequential()

    # Creating parameters grid
    units = hp.Int('neurons', 8, 128, step = 8)

    # Building model architecture
    model.add(Input(shape=(8,)))
    model.add(Dense(units = units, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    # Compiling the model
    model.compile('adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

    # Returning the model
    return model

# This is GridSearchCV of sklearn in keras
tuner = kt.RandomSearch(
    build_model, # Your function
    objective = 'val_accuracy', # What you want to maximize(metrics) or minimize(loss functions)
    max_trials = 5, # number of epochs for each combination of parameters
    directory = 'Hyperparameter Tuning', # Store all the results in this folder
    project_name = 'number of neurons' # path of folder inside the above mentioned directory. Stores in JSON format
)

# Fitting and Searching
tuner.search(X_train, y_train, validation_data = (X_test, y_test), epochs = 5)

Trial 5 Complete [00h 00m 04s]
val_accuracy: 0.6883116960525513

Best val_accuracy So Far: 0.7922077775001526
Total elapsed time: 00h 00m 21s


In [75]:
tuner.get_best_hyperparameters()[0].values

{'neurons': 112}

In [76]:
model = tuner.get_best_models(num_models = 1)[0]

/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [77]:
history = model.fit(X_train, y_train, validation_data = (X_test, y_test), epochs = 100, initial_epoch=5)

Epoch 6/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 3s 72ms/step - accuracy: 0.7435 - loss: 0.5308 - val_accuracy: 0.7857 - val_loss: 0.5013
Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7960 - loss: 0.4691 - val_accuracy: 0.7922 - val_loss: 0.4867
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7967 - loss: 0.4550 - val_accuracy: 0.7857 - val_loss: 0.4865
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7773 - loss: 0.4597 - val_accuracy: 0.7792 - val_loss: 0.4884
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7878 - loss: 0.4723 - val_accuracy: 0.7727 - val_loss: 0.4923
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7918 - loss: 0.4459 - val_accuracy: 0.7662 - val_loss: 0.4920
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7831 - loss: 0.4458 - val_accuracy: 0.7597 - val_loss: 0.4971
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8143 - loss: 0.4095 - val_accuracy: 0.759

In [78]:
# Selecting optimum number of layers
def build_model(hp):
    # Building model architecture
    model = Sequential()

    # Building model architecture
    model.add(Input(shape=(8,)))
    for i in range(hp.Int('num_layers', min_value = 1, max_value = 10)):
        model.add(Dense(72, activation = 'relu'))
    model.add(Dense(1, activation='sigmoid'))

    # Compiling the model
    model.compile('adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

    # Returning the model
    return model

# This is GridSearchCV of sklearn in keras
tuner = kt.RandomSearch(
    build_model, # Your function
    objective = 'val_accuracy', # What you want to maximize(metrics) or minimize(loss functions)
    max_trials = 5, # number of epochs for each combination of parameters
    directory = 'Hyperparameter Tuning', # Store all the results in this folder
    project_name = 'Number of Layers' # path of folder inside the above mentioned directory. Stores in JSON format
)

# Fitting and Searching
tuner.search(X_train, y_train, validation_data = (X_test, y_test), epochs = 5)

Trial 5 Complete [00h 00m 07s]
val_accuracy: 0.7532467246055603

Best val_accuracy So Far: 0.8051947951316833
Total elapsed time: 00h 00m 32s


In [80]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 2}

In [81]:
model = tuner.get_best_models(num_models = 1)[0]

/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 14 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [82]:
history = model.fit(X_train, y_train, validation_data = (X_test, y_test), epochs = 100, initial_epoch=5)

Epoch 6/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 44ms/step - accuracy: 0.7736 - loss: 0.5176 - val_accuracy: 0.7662 - val_loss: 0.5073
Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.7969 - loss: 0.4634 - val_accuracy: 0.7922 - val_loss: 0.4940
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7817 - loss: 0.4460 - val_accuracy: 0.7727 - val_loss: 0.5008
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7957 - loss: 0.4371 - val_accuracy: 0.7468 - val_loss: 0.5097
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8060 - loss: 0.4041 - val_accuracy: 0.7662 - val_loss: 0.5054
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8140 - loss: 0.4117 - val_accuracy: 0.7468 - val_loss: 0.5175
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7844 - loss: 0.4393 - val_accuracy: 0.7532 - val_loss: 0.5183
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8005 - loss: 0.4235 - val_accuracy: 0.753

In [83]:
# Consolidating
def build_model(hp):
    model = Sequential()
    model.add(Input(shape = (8, )))

    for i in range(hp.Int('num_layers', min_value = 1, max_value = 10)):
        units = hp.Int(f"units-{i}", min_value = 8, max_value = 128, step = 8)
        activations = hp.Choice(f'activation-{i}', values = ['relu', 'tanh', 'sigmoid'])
        rate = hp.Choice(f'rate-{i}', values = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])
        model.add(Dense(
                units = units,
                activation = activations
            )
        )
        model.add(Dropout(
                rate = rate
            )
        )

    model.add(Dense(1, activation = 'sigmoid'))
    optimizers = hp.Choice('optimizer', values = ['adam', 'sgd', 'adadelta', 'adagrad'])

    model.compile(optimizer = optimizers, loss = 'binary_crossentropy', metrics = ['accuracy'])

    return model

tuner = kt.RandomSearch(
    build_model,
    objective = 'val_accuracy',
    max_trials = 5,
    directory = 'Hyperparameter Tuning',
    project_name = 'Combine all'
)

tuner.search(X_train, y_train, validation_data = (X_test, y_test), epochs = 5, verbose = 1)

Trial 5 Complete [00h 00m 05s]
val_accuracy: 0.6428571343421936

Best val_accuracy So Far: 0.6428571343421936
Total elapsed time: 00h 00m 57s


In [84]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 8,
 'units-0': 112,
 'activation-0': 'relu',
 'rate-0': 0.9,
 'optimizer': 'adam',
 'units-1': 32,
 'activation-1': 'relu',
 'rate-1': 0.1,
 'units-2': 16,
 'activation-2': 'tanh',
 'rate-2': 0.7,
 'units-3': 8,
 'activation-3': 'relu',
 'rate-3': 0.1,
 'units-4': 8,
 'activation-4': 'relu',
 'rate-4': 0.1,
 'units-5': 8,
 'activation-5': 'relu',
 'rate-5': 0.1,
 'units-6': 8,
 'activation-6': 'relu',
 'rate-6': 0.1,
 'units-7': 8,
 'activation-7': 'relu',
 'rate-7': 0.1}

In [85]:
model = tuner.get_best_models(num_models = 1)[0]

/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 38 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [86]:
history = model.fit(X_train, y_train, validation_data = (X_test, y_test), epochs = 100, initial_epoch = 5)

Epoch 6/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 10s 215ms/step - accuracy: 0.6557 - loss: 0.6620 - val_accuracy: 0.6429 - val_loss: 0.6733
Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.6270 - loss: 0.6690 - val_accuracy: 0.6429 - val_loss: 0.6653
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6498 - loss: 0.6675 - val_accuracy: 0.6429 - val_loss: 0.6606
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6691 - loss: 0.6725 - val_accuracy: 0.6429 - val_loss: 0.6616
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6171 - loss: 0.6755 - val_accuracy: 0.6429 - val_loss: 0.6607
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6425 - loss: 0.6581 - val_accuracy: 0.6429 - val_loss: 0.6563
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6536 - loss: 0.6455 - val_accuracy: 0.6429 - val_loss: 0.6496
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6368 - loss: 0.6635 - val_accuracy: 0.6

In [87]:
def build_model(hp):
    model = Sequential()

    # Define hyperparameter for number of units in a Dense layer
    hp_units = hp.Choice('units', values=[32, 64]) # Discrete choices

    # Define hyperparameter for activation function
    hp_activation = hp.Choice('activation', values=['relu', 'tanh']) # Discrete choices

    # Define hyperparameter for learning rate
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3]) # Discrete choices

    model.add(Dense(units=hp_units, activation=hp_activation))
    model.add(Dense(1, activation='sigmoid')) # Example output layer

    model.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    return model

# Instantiate the GridSearch tuner
tuner = kt.GridSearch(
    build_model,
    objective='val_accuracy',
    directory='my_grid_search_dir',
    project_name='grid_hp_test'
)

# Run the search
tuner.search(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

Trial 8 Complete [00h 00m 04s]
val_accuracy: 0.7792207598686218

Best val_accuracy So Far: 0.7922077775001526
Total elapsed time: 00h 00m 44s


In [88]:
# Get the summary of the search space
tuner.search_space_summary()

Search space summary
Default search space size: 3
units (Choice)
{'default': 32, 'conditions': [], 'values': [32, 64], 'ordered': True}
activation (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'tanh'], 'ordered': False}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001], 'ordered': True}
